In [2]:
import pandas as pd
dfg = pd.read_csv ('george_courses.csv')
print(dfg)

                                Course              City      State Country  \
0                             Wynlakes        Montgomery    Alabama      US   
1                          Lagoon Park        Montgomery    Alabama      US   
2    University of Alabama Golf Course        Tuscaloosa    Alabama      US   
3                       Hidden Meadows          Nortport    Alabama      US   
4                           Bent Brook          Bessemer    Alabama      US   
..                                 ...               ...        ...     ...   
154                            Gateway        Montgomery    Alabama      US   
155               Mohansic Golf Course  Yorktown Heights   New York      US   
156        Dead Horse Lake Golf Course         Knoxville  Tennessee      US   
157                     Farm Golf Club        Rocky Face    Georgia      US   
158  Chattanooga Golf and Country Club       Chattanooga  Tennessee      US   

     Year  
0    1990  
1    1996  
2    1997  
3  

In [3]:
import sqlite3
cnx = sqlite3.connect('garmin.db')

df_db = pd.read_sql_query("SELECT * FROM courses", cnx)
print(df_db.head())

   id                       name                 address            city  \
0   1       Big Spring Lake Golf     860 Country Club Rd     Albertville   
1   2   Willow Brook Golf Course              134 Par Dr     Albertville   
2   3  Willow Point Country Club    2544 Willow Point Rd  Alexander City   
3   4    Aliceville Country Club          285 Fairway Ln      Aliceville   
4   5       Alpine Bay Golf Club         9855 Renfroe Rd          Alpine   

     state        country   latitude  longitude  
0  Alabama  United States  34.297449 -86.199218  
1  Alabama  United States  34.352559 -86.119746  
2  Alabama  United States  32.786363 -85.976556  
3  Alabama  United States  33.303987 -88.058330  
4  Alabama  United States  33.436113 -86.299047  


In [4]:
import fuzzymatcher
g_left = ["Course", "City", "State"]
db_right = ["name", "city", "state"]
fuzzy_match = fuzzymatcher.fuzzy_left_join(dfg, df_db, g_left, db_right)
fuzzy_match.to_csv('george_fuzzy_match.csv')
fuzzy_match

,best_match_score,__id_left,__id_right,Course,City,State,Country,Year,id,name,address,city,state,country,latitude,longitude
0,0.766885,0_left,211_right,Wynlakes,Montgomery,Alabama,US,1990,212,Wynlakes Golf & Country Club,7900 Wynlakes Blvd,Montgomery,Alabama,United States,32.349395,-86.150140
1,0.693327,1_left,209_right,Lagoon Park,Montgomery,Alabama,US,1996,210,Lagoon Park Golf Course,2855 Lagoon Park Dr,Montgomery,Alabama,United States,32.406803,-86.221579
2,0.141090,2_left,97_right,University of Alabama Golf Course,Tuscaloosa,Alabama,US,1997,98,Country Club of Alabama,101 Merion Dr,Eufaula,Alabama,United States,31.819869,-85.160930
109,0.586327,3_left,217_right,Hidden Meadows,Nortport,Alabama,US,1998,218,Hidden Meadows Golf Course,13000 Old Cove Rd,Northport,Alabama,United States,33.296915,-87.638751
111,-0.012171,4_left,29_right,Bent Brook,Bessemer,Alabama,US,1999,30,Woodward Golf Club,2141 Wheel Dr,Bessemer,Alabama,United States,33.428202,-86.955865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,0.459786,154_left,208_right,Gateway,Montgomery,Alabama,US,2022,209,Gateway Park Golf Course,3800 Davenport Pkwy,Montgomery,Alabama,United States,32.332937,-86.329099
2692,1.159750,155_left,12257_right,Mohansic Golf Course,Yorktown Heights,New York,US,2022,12258,Mohansic Golf Course,1500 BALDWIN RD,Yorktown Heights,New York,United States,41.266376,-73.806395
2693,1.032789,156_left,15929_right,Dead Horse Lake Golf Course,Knoxville,Tennessee,US,2022,15930,Dead Horse Lake Golf Club,555 Frank Gardner Lane,Knoxville,Tennessee,United States,35.917030,-84.112238
2821,0.938531,157_left,4500_right,Farm Golf Club,Rocky Face,Georgia,US,2022,4501,The Farm Golf Club,187 S Goosehill Rd,Rocky Face,Georgia,United States,34.772147,-85.021965


In [4]:
import pandas as pd
import folium

dfm = pd.read_csv('george_fuzzy_match.csv')
print(dfm)
courses_to_map = []
for i in dfm.itertuples():
    # if i[2] < .2:
    #     print(f"course: {i[5]}, match score:{i[2]}")  # list bad matches
    if i[2] > .2:
        c = {
            'course':i[11].lstrip(),
            'lat':i[16],
            'long':i[17]
        }
        courses_to_map.append(c)

cm = dfm[["name","latitude","longitude", "best_match_score"]]
map = folium.Map(location=[cm.latitude.mean(), cm.longitude.mean()], zoom_start=3, control_scale=True)
map

     Unnamed: 0  best_match_score __id_left   __id_right  \
0             0          0.766885    0_left    211_right   
1             1          0.693327    1_left    209_right   
2             2          0.141090    2_left     97_right   
3           109          0.586327    3_left    217_right   
4           111         -0.012171    4_left     29_right   
..          ...               ...       ...          ...   
154        2737          0.459786  154_left    208_right   
155        2738          1.159750  155_left  12257_right   
156        2739          1.032789  156_left  15929_right   
157        2867          0.938531  157_left   4500_right   
158        2868          0.767903  158_left  15785_right   

                                Course              City      State Country  \
0                             Wynlakes        Montgomery    Alabama      US   
1                          Lagoon Park        Montgomery    Alabama      US   
2    University of Alabama Golf Course    